In [3]:
# Install dependencies
%pip install langchain langchain-google-genai chromadb python-dotenv unstructured --quiet



Note: you may need to restart the kernel to use updated packages.


In [5]:
# Setup environment and load API key

import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
if not GOOGLE_API_KEY:
    raise RuntimeError("Set GOOGLE_API_KEY in your .env file")


In [ ]:
# Imports

from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate


In [ ]:
DATA_PATH = "alice_in_wonderland.md"  # Updated file extension to .md
PERSIST_DIR = "chroma_db_gemini"
COLLECTION = "docs"

CHUNK_SIZE = 1000
CHUNK_OVERLAP = 500

# Using UnstructuredMarkdownLoader for the markdown file
loader = UnstructuredMarkdownLoader(DATA_PATH)
docs = loader.load()
print(f"Loaded {len(docs)} documents")

FileNotFoundError: Directory not found: 'alice_in_wonderland.txt'